In [8]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor,RandomHorizontalFlip,Compose,Grayscale
from torch.utils.data.dataloader import DataLoader

root = "../data/"

train_ds = ImageFolder(root+"train", Compose([Grayscale(num_output_channels=1),RandomHorizontalFlip(),ToTensor()]))
train_dl = DataLoader(train_ds, 100, shuffle=True, pin_memory=True)

val_ds = ImageFolder(root+"val", ToTensor())
val_dl = DataLoader(val_ds, 70, pin_memory=True)

print(len(train_ds))
img_cls = train_ds.classes

CLS_SIZE = len(img_cls)
print(img_cls)
print(train_ds[0][0].shape)

182000
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
torch.Size([1, 256, 256])


In [9]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [10]:
import torch.nn as nn

class ResNet12(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1,15,kernel_size=3,stride=2,padding=1)
        self.conv2 = nn.Conv2d(15,30,kernel_size=3,stride=1,padding=1)
        self.conv3 = nn.Conv2d(30,60,kernel_size=3,stride=2,padding=1)
        self.conv4 = nn.Conv2d(60,120,kernel_size=3,stride=1,padding=1)
        
        self.conv5 = nn.Conv2d(120,200,kernel_size=3,stride=2,padding=1)
        self.conv6 = nn.Conv2d(200,200,kernel_size=3,stride=1,padding=1)
        self.conv7 = nn.Conv2d(200,200,kernel_size=3,stride=2,padding=1)

        self.conv8 = nn.Conv2d(200,360,kernel_size=3,stride=1,padding=1)
        self.conv9 = nn.Conv2d(360,360,kernel_size=3,stride=2,padding=1)
        self.conv10 = nn.Conv2d(360,360,kernel_size=3,stride=1,padding=1)
        
        self.pool = nn.MaxPool2d(2,2)
        self.avgpool = nn.AvgPool2d(2,2)
        self.flat = nn.Flatten()
        
        self.linear = nn.Linear(360,CLS_SIZE)
    

    def forward(self, data):
        # data 1 256 256

        out = torch.relu(self.conv1(data))#15 128 128
        out = torch.relu(self.conv2(out))#30 128 128
        
        out = torch.relu(self.conv3(out))#60 64 64

        out = torch.relu(self.conv4(out))#120 128 128
        
        out = torch.relu(self.conv5(out))#200 64 64
        out = torch.relu(self.conv6(out))#200 64 64
        out = torch.relu(self.conv7(out))#200 64 64
        
        out = torch.relu(self.conv8(out))#360 32 32
        out = torch.relu(self.conv9(out))#360 32 32
        out = torch.relu(self.conv10(out))#360 32 32
        
        
        out = self.avgpool(out)#360 16 16
        out = self.avgpool(out)#360 8 8
        out = self.avgpool(out)#360 4 4
        out = self.avgpool(out)#360 2 2
        out = self.avgpool(out)#360 1 1
        
        out = self.flat(out)#360
        out = self.linear(out)#10
        
        out = torch.softmax(out,dim=-1)
        
        return out


In [11]:
sign_model = ResNet12().to(device)
loss_fn = nn.BCELoss()
opt = torch.optim.Adam(sign_model.parameters(),lr=0.001)

In [12]:
import numpy as np

def fit(epochs):
    for epoch in range(epochs):
        print("Epoch: ",epoch+1)
        x = 0
        for images,labels in train_dl:
            x += 1
            preds = sign_model(images.to(device))

            targets = torch.zeros(BATCH_SIZE,CLS_SIZE, dtype=torch.float32)
            
            for i in range(BATCH_SIZE):
                targets[i,label_batch[i]]=1

            targets = torch.tensor(targets)

            loss = loss_fn(preds.to(device), targets.to(device))
            loss.backward()
            
            opt.step()
            opt.zero_grad()

            print(x,"/",len(train_dl))

        print("Loss: ",round(loss.item(),6))

        with torch.no_grad():
            correct = 0
            for images,labels in test_dl:
                preds = sign_model(images.to(device))

                for i in range(len(preds)):
                    if (preds[i].max()==preds[i][labels[i].item()]):
                        correct += 1
            
            acc = correct/len(test_ds)
            print("Accuracy: ",round(acc*100,3))
            print("")
        if acc>0.9:
            break

fit(1)

Epoch:  1


NameError: name 'BATCH_SIZE' is not defined

In [ ]:
# model = ResNet12()
# try:
#     if torch.cuda.is_available():
#         model.load_state_dict(torch.load("./saved_models/Cifar10ResNet12.pth"))
#         print("Model Loaded(GPU)")
#     else:
#         model.load_state_dict(torch.load("./saved_models/Cifar10ResNet12.pth",map_location=torch.device("cpu")))
#         print("Model Loaded(CPU)")
# except Exception as e:
#     print(e)
#     print("Error Occured in loading saved weights, training for new weights")
# fit(20)

In [ ]:
# from PIL import Image
# import matplotlib.pyplot as plt

# def predict(path,model):
#     img = Image.open(path)
    
#     if img.size != (32,32):
#         img = img.resize((32,32))
    
#     img_arr = np.asarray(img)
#     plt.imshow(img_arr)
    
#     img_arr = img_arr/255
#     img_tsr = torch.Tensor([img_arr])
#     img_tsr = img_tsr.permute(0,3,1,2)
    
#     for i in range(3):
#         img_tsr[0][i] = (img_tsr[0][i]-mean[i])/dev[i] 
    
#     pred = model(img_tsr).detach()
#     pred = np.array(pred[0])
#     pred_index = np.where(pred==max(pred))[0][0]
    
#     print("Prediction: ",img_cls[pred_index])